# Deterministic geographical model

## Global variables to use

In [1]:
limit_to_england = False
onset_to_ambulance_arrival = 60
arrival_to_ivt = 40
arrival_to_mt = 90
net_operational_delay_to_mt_for_transfer = 60

file_name = 'lsoa_base'

## Load packages

In [2]:
import numpy as np
import pandas as pd

from utilities.clinical_outcome import Clinical_outcome

import warnings
warnings.filterwarnings("ignore")

## Load and parse data

### Load and parse hospital data

In [3]:

hospitals = pd.read_csv('./data/stroke_hospitals_2022.csv')
hospitals['Use'] = hospitals[['Use_IVT', 'Use_MT']].max(axis=1)
mask = hospitals['Use'] == 1
hospitals = hospitals[mask]
if limit_to_england:
    mask = hospitals['Country'] == 'England'
    hospitals = hospitals[mask]

### Load and parse LSOA travel matrix

In [4]:
# Load data
admissions = pd.read_csv(
    './data/lsoa_predicted_admissions.csv', index_col='lsoa11nm')
travel_matrix = pd.read_csv(
    './data/lsoa_travel_time_matrix_calibrated.csv', index_col='LSOA')

if limit_to_england:
    england_lsoa = list(admissions[admissions['England']==1].index)
    admissions = admissions.loc[england_lsoa]
    travel_matrix = travel_matrix.loc[england_lsoa]

# Identify closest IVT unit
ivt_hospitals = list(hospitals[hospitals['Use_IVT']==1]['Postcode'])
lsoa_ivt_travel_time = dict(travel_matrix[ivt_hospitals].min(axis=1))
lsoa_ivt_unit = dict(travel_matrix[ivt_hospitals].idxmin(axis=1))

# Identify closest MT Unit
mt_hospitals = list(hospitals[hospitals['Use_MT']==1]['Postcode'])
lsoa_mt_travel_time = dict(travel_matrix[mt_hospitals].min(axis=1))
lsoa_mt_unit = dict(travel_matrix[mt_hospitals].idxmin(axis=1))

# Identify closest MSU base
msu_hospitals = list(hospitals[hospitals['Use_MSU']==1]['Postcode'])
lsoa_msu_travel_time = dict(travel_matrix[msu_hospitals].min(axis=1))
lsoa_msu_unit = dict(travel_matrix[msu_hospitals].idxmin(axis=1))

### Load and parse inter_hospital travel time for MT

In [5]:
# Load data
inter_hospital_time = pd.read_csv(
    './data/inter_hospital_time_calibrated.csv', index_col='from_postcode')

# Restrict to IVT and MSU hospitals in use
inter_hospital_time = inter_hospital_time.loc[ivt_hospitals][mt_hospitals]

# Set distance between same hospitals to zero
rows = list(inter_hospital_time.index)
cols = list(inter_hospital_time)
for row in rows:
    for col in cols:
        if row == col:
            inter_hospital_time.loc[row][col] = 0

# Identify closest transfer unit
mt_transfer_time = dict(inter_hospital_time.min(axis=1))
mt_transfer_unit = dict(inter_hospital_time.idxmin(axis=1))

## Calculate results

In [6]:
# Set up table for results
lsoa_list = list(lsoa_ivt_unit.keys())
results = pd.DataFrame(index=lsoa_list)

# Collate info
results['closest_ivt_unit'] = [lsoa_ivt_unit[x] for x in lsoa_list]
results['closest_ivt_time'] = [lsoa_ivt_travel_time[x] for x in lsoa_list]
results['closest_mt_unit'] = [lsoa_mt_unit[x] for x in lsoa_list]
results['closest_mt_time'] = [lsoa_mt_travel_time[x] for x in lsoa_list]
results['transfer_mt_unit'] = [mt_transfer_unit[x] for x in results['closest_ivt_unit']]
results['transfer_mt_time'] = [mt_transfer_time[x] for x in results['closest_ivt_unit']]
results['mt_transfer_required'] = results['closest_ivt_unit'] != results['closest_mt_unit']
results['msu_unit'] = [lsoa_msu_unit[x] for x in lsoa_list]
results['msu_time'] = [lsoa_msu_travel_time[x] for x in lsoa_list]

# Set up lists for total times to treatment
ivt_drip_ship = []
ivt_mothership = []
mt_drip_ship = []
mt_mothership = []

# Calculate total times for treatment
for lsoa in lsoa_list:
    # Time to IVT for drip and ship
    ivt_drip_ship.append(
        onset_to_ambulance_arrival + results['closest_ivt_time'][lsoa] + arrival_to_ivt)
    # Time to IVT for mothership
    ivt_mothership.append(
        onset_to_ambulance_arrival + results['closest_mt_time'][lsoa] + arrival_to_ivt)
    # Time to MT for drip and ship
    mt_drip_ship.append(
        onset_to_ambulance_arrival + results['closest_ivt_time'][lsoa] + 
        results['transfer_mt_time'][lsoa] + (net_operational_delay_to_mt_for_transfer * 
        results['mt_transfer_required'][lsoa]) + arrival_to_mt)
    # Time to MT for mothership
    mt_mothership.append(
        onset_to_ambulance_arrival +  results['closest_mt_time'][lsoa] + arrival_to_mt)
    
results['ivt_drip_ship'] = ivt_drip_ship
results['ivt_mothership'] = ivt_mothership
results['mt_drip_ship'] = mt_drip_ship
results['mt_mothership'] = mt_mothership

# Sort
results.sort_index(inplace=True)

# Show results
results.head().T

,Adur 001A,Adur 001B,Adur 001C,Adur 001D,Adur 001E
closest_ivt_unit,BN25BE,BN25BE,BN112DH,BN112DH,BN112DH
closest_ivt_time,17.6,18.7,17.6,17.6,16.5
closest_mt_unit,BN25BE,BN25BE,BN25BE,BN25BE,BN25BE
closest_mt_time,17.6,18.7,19.8,19.8,19.8
transfer_mt_unit,BN25BE,BN25BE,BN25BE,BN25BE,BN25BE
transfer_mt_time,0.0,0.0,31.6,31.6,31.6
mt_transfer_required,False,False,True,True,True
msu_unit,BN25BE,BN25BE,BN25BE,BN25BE,BN25BE
msu_time,17.6,18.7,19.8,19.8,19.8
ivt_drip_ship,117.6,118.7,117.6,117.6,116.5


## Add outcomes

In [7]:
# Load mRS distributions
mrs_dists = pd.read_csv(
    './utilities/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


In [8]:
outcome_model = Clinical_outcome(mrs_dists)

Demo of outcome model

In [9]:
time_to_ivt = 90
time_to_mt = 120
outcomes = outcome_model.calculate_outcomes(
    time_to_ivt, time_to_mt, patients=100, random_spacing=False)

outcomes

{'lvo_untreated_probs': array([0.05, 0.08, 0.14, 0.16, 0.24, 0.14, 0.19]),
 'nlvo_untreated_probs': array([0.2 , 0.26, 0.12, 0.13, 0.14, 0.06, 0.09]),
 'lvo_ivt_probs': array([0.1 , 0.08, 0.13, 0.16, 0.21, 0.12, 0.2 ]),
 'lvo_mt_probs': array([0.21, 0.13, 0.13, 0.17, 0.17, 0.08, 0.11]),
 'nlvo_ivt_probs': array([0.37, 0.22, 0.11, 0.12, 0.09, 0.03, 0.06]),
 'lvo_untreated_mean_utility': 0.33190000000000003,
 'nlvo_untreated_mean_utility': 0.5997000000000001,
 'lvo_ivt_mean_utility': 0.3708,
 'lvo_mt_mean_utility': 0.5266000000000001,
 'nlvo_ivt_mean_utility': 0.7121999999999999,
 'lvo_ivt_added_utility': 0.03889999999999999,
 'lvo_mt_added_utility': 0.19470000000000004,
 'nlvo_ivt_added_utility': 0.11249999999999982,
 'lvo_untreated_cum_probs': array([0.05, 0.13, 0.27, 0.43, 0.67, 0.81, 1.  ]),
 'nlvo_untreated_cum_probs': array([0.2 , 0.46, 0.58, 0.71, 0.85, 0.91, 1.  ]),
 'lvo_ivt_cum_probs': array([0.1 , 0.18, 0.31, 0.47, 0.68, 0.8 , 1.  ]),
 'lvo_mt_cum_probs': array([0.21, 0.34, 0.

In [10]:
def get_drip_ship_outcomes(row):
    """ Calculate outcomes based on drip and ship times to IVT and MT"""
    time_to_ivt = row['ivt_drip_ship']
    time_to_mt = row['mt_drip_ship']
    outcomes = outcome_model.calculate_outcomes(
        time_to_ivt, time_to_mt, patients=100, random_spacing=False)

    return outcomes

def get_mothership_outcomes(row):
    """ Calculate outcomes based on drip and ship times to IVT and MT"""
    time_to_ivt = row['ivt_mothership']
    time_to_mt = row['mt_mothership']
    outcomes = outcome_model.calculate_outcomes(
        time_to_ivt, time_to_mt, patients=100, random_spacing=False)

    return outcomes

In [11]:
# Get drip and ship results
outcomes = results.apply(get_drip_ship_outcomes, axis=1)
# Reformat
outcomes = list(outcomes.values)
outcomes = pd.DataFrame(outcomes)
colnames = list(outcomes)
new_colnames = ['drip_ship_' + colname for colname in colnames]
outcomes = pd.DataFrame(outcomes.values, columns = new_colnames, index=results.index)
# Add to results
results = pd.concat([results, outcomes], axis=1)

# Get mothership results
outcomes = results.apply(get_mothership_outcomes, axis=1)
# Reformat
outcomes = list(outcomes.values)
outcomes = pd.DataFrame(outcomes)
colnames = list(outcomes)
new_colnames = ['mothership_' + colname for colname in colnames]
outcomes = pd.DataFrame(outcomes.values, columns = new_colnames, index=results.index)
# Add to results
results = pd.concat([results, outcomes], axis=1)

In [12]:
# Add admissions to results

results = results.merge(admissions['LSOA_predicted_admissions'],
                    how='left', left_index=True, right_index=True)

In [13]:
# Save
results.to_csv(f'./output/{file_name}.csv', index_label='lsoa')